In [6]:
from langchain_chroma import Chroma
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter, Language
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_core.documents import Document
from typing import List
import uuid
import subprocess
import socket
import json
import re

In [7]:
# Get Windows host IP from WSL
def get_windows_host_ip():
    # Method 1: Try WSL2 bridge interface (default route)
    try:
        result = subprocess.run(
            ["ip", "route", "show", "default"],
            capture_output=True,
            text=True
        )
        for line in result.stdout.split('\n'):
            if 'default' in line and 'via' in line:
                parts = line.split()
                via_index = parts.index('via')
                ip = parts[via_index + 1]
                # Test if Ollama is accessible at this IP
                try:
                    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                    sock.settimeout(1)
                    sock.connect((ip, 11434))
                    sock.close()
                    print(f"✓ Found Ollama at {ip} (WSL default gateway)")
                    return ip
                except:
                    print(f"✗ Port 11434 not accessible at {ip}")
    except Exception as e:
        print(f"✗ Method 1 failed: {e}")
    
    # Method 2: Try /etc/resolv.conf nameserver
    try:
        result = subprocess.run(
            ["cat", "/etc/resolv.conf"],
            capture_output=True,
            text=True
        )
        for line in result.stdout.split('\n'):
            if 'nameserver' in line:
                ip = line.split()[1]
                # Test if Ollama is accessible at this IP
                try:
                    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                    sock.settimeout(1)
                    sock.connect((ip, 11434))
                    sock.close()
                    print(f"✓ Found Ollama at {ip} (resolv.conf)")
                    return ip
                except:
                    print(f"✗ Port 11434 not accessible at {ip}")
    except Exception as e:
        print(f"✗ Method 2 failed: {e}")
    
    # Fallback to localhost
    print("⚠ Falling back to localhost")
    return "127.0.0.1"

def markdown_to_document(file_path: str, chunk_size: int, chunk_overlap: int, is_execute: bool = False) -> List[Document]:
    if is_execute:
        # Load metadata
        metadata_loader = UnstructuredMarkdownLoader(file_path, mode="elements")
        metadata_docs = metadata_loader.load()
        
        # Load content
        with open(file_path) as f:
            file = f.read()
        text_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.MARKDOWN, chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        documents = text_splitter.create_documents([file], [metadata_docs[0].metadata])

        
        return documents

def combine_list_metadata(document: Document, key_to_combine: str, is_execute: bool = False):
    if is_execute:
        langs = document.metadata.get(key_to_combine)
        if langs is None:
            document.metadata[key_to_combine] = ""
        elif isinstance(langs, str):
            pass
        else:
            try:
                document.metadata[key_to_combine] = ",".join(map(str, langs))
            except TypeError:
                import json
                document.metadata[key_to_combine] = json.dumps(langs)

def add_documents(vector_store: Chroma, documents: List[Document], is_execute: bool = False):
    if is_execute:
        ids = [str(uuid.uuid4()) for _ in documents]
        vector_store.add_documents(documents=documents, ids=ids)

In [ ]:
WINDOWS_HOST_IP = get_windows_host_ip()

EMBEDDING = OllamaEmbeddings(base_url=f"http://{WINDOWS_HOST_IP}:11434",
                             model="embeddinggemma:300m")
vector_store_zh = Chroma(
    collection_name="eda-hospital-zh",
    embedding_function=EMBEDDING,
    persist_directory="./vectordb/eda-hospital-zh",
    create_collection_if_not_exists=False,
)
vector_store_en = Chroma(
    collection_name="eda-hospital-en",
    embedding_function=EMBEDDING,
    persist_directory="./vectordb/eda-hospital-en",
    create_collection_if_not_exists=False,
)

✓ Found Ollama at 172.17.96.1 (WSL default gateway)


In [ ]:
documents = []

for folder in os.listdir("./assets/output-en"):
    for file in os.listdir(f"./assets/output-en/{folder}"):
        if file.endswith(".md"):
            file_path = f"./assets/output-en/{folder}/{file}"
            document = markdown_to_document(file_path=file_path,
                                 chunk_size=2048,
                                 chunk_overlap=200,
                                 is_execute=False)
            if document is not None:
                documents.extend(document)

for document in documents:
    combine_list_metadata(document=document, key_to_combine="languages", is_execute=False)

add_documents(vector_store=vector_store_en, documents=documents, is_execute=False)

In [10]:
vector_store_en.get().keys()

dict_keys(['ids', 'embeddings', 'documents', 'uris', 'included', 'data', 'metadatas'])

In [12]:
len(vector_store_zh.get()["documents"])

50

# Search Database

In [108]:
with open("./filename_translation.json", "r") as f:
    filename_translation = json.load(f)

In [122]:
docs = vector_store_en.similarity_search_with_relevance_scores(query="Nosebleed", k=5, score_threshold=0)
for doc in docs:
    if doc[1] > 0:
        print("--Original retrieved files in English")
        print(f'{doc[0].metadata["filename"]} | {doc[1]}')
        print("--Chinese")
        base_filename = re.sub(r'(?i)(?:_\d+)?\.md$', '', doc[0].metadata["filename"])  # strip trailing _<digits>.md (case-insensitive)
        print(next((k for k, v in filename_translation.items() if v == base_filename), base_filename))
        print("\n")

--Original retrieved files in English
HA-7-0037(1)_Nosebleed Precautions.md | 0.48818418172395495
--Chinese
HA-7-0037(1)_流鼻血注意事項


--Original retrieved files in English
HA-7-0042(1)_Chest Contusion Precautions.md | 0.14118909364006738
--Chinese
HA-7-0042(1)_胸部挫傷注意事項


--Original retrieved files in English
HA-7-0029(1)_Patients with difficulty breathing and their family members should.md | 0.06559223022110827
--Chinese
HA-7-0029(1)_呼吸困難病患及家屬應


--Original retrieved files in English
HA-7-0040(1)_Precautions for Peptic Ulcer.md | 0.03593939348278363
--Chinese
HA-7-0040(1)_消化性潰瘍注意事項




/tmp/ipykernel_206462/3112790189.py:1: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='e45e1586-fbc1-449c-9b50-fab7c03919ac', metadata={'last_modified': '2025-11-07T22:56:08', 'filetype': 'text/markdown', 'category_depth': 1, 'category': 'Title', 'source': './assets/output-en/emergency-急診/HA-7-0037(1)_Nosebleed Precautions.md', 'file_directory': './assets/output-en/emergency-急診', 'filename': 'HA-7-0037(1)_Nosebleed Precautions.md', 'element_id': 'eee42f77676d53a3303600a5570a8f0c', 'languages': 'eng'}, page_content="## Nosebleed Precautions\n\n1. Avoid prolonged exposure to air-conditioned rooms or picking the nose, to prevent nasal mucosa from becoming dry and cracked, which may lead to nosebleeds or infections.\n\n2. When experiencing a nosebleed, the priority is to stop the bleeding. First, have the patient sit upright. If necessary, insert a piece of tissue into the nostril. However, if bleeding cannot be controlled, apply pressure to the bridge of the nose

In [121]:
docs = vector_store_zh.similarity_search_with_relevance_scores(query="流鼻血", k=5, score_threshold=0)
for doc in docs:
    if doc[1] > 0:
        print("--Original retrieved files in Chinese")
        print(f'{doc[0].metadata["filename"]} | {doc[1]}')
        print("--English")
        base_filename = re.sub(r'(?i)(?:_\d+)?\.md$', '', doc[0].metadata["filename"])  # strip trailing _<digits>.md (case-insensitive)
        print(filename_translation.get(base_filename, base_filename))
        print("\n")

--Original retrieved files in Chinese
HA-7-0037(1)_流鼻血注意事項_0.md | 0.36574446793301796
--English
HA-7-0037(1)_Nosebleed Precautions




/tmp/ipykernel_206462/1115547260.py:1: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='1dc54bd1-107f-4943-b61b-dcdc50309f78', metadata={'languages': 'kor', 'file_directory': './assets/output-zh/emergency-急診', 'category_depth': 1, 'element_id': 'dd0a44ee20d5988d7bd3ef072682a39f', 'last_modified': '2025-11-06T15:14:22', 'category': 'Title', 'filename': 'HA-7-0037(1)_流鼻血注意事項_0.md', 'source': './assets/output-zh/emergency-急診/HA-7-0037(1)_流鼻血注意事項_0.md', 'filetype': 'text/markdown'}, page_content='## 流鼻血注意事项\n\n1. 避免長期處於冷氣房內或挖鼻孔，防止鼻黏膜乾燥破皮，導致流鼻血或感染情形。\n\n2. 流鼻血時以止血為優先處理原則。請先讓病患採坐姿，必要時可塞入衛生紙條於鼻孔內，但若仍無法止血，可於鼻樑處加壓或冰敷（創傷病患，不可隨意塞住鼻孔，以防呼吸道血液嗆入窒息）。\n\n3. 高血壓病患若測量出高於平時的血壓數值，可服用醫師指示之急用降血壓藥，並保持休息狀態，避免過度用力行為及走動，以半坐臥為原則，並予以止血。\n\n4. 請配合急診醫師建議用藥，及相關藥物服用注意事項；就醫後鼻內置入之填塞紗布棉條，請勿隨意移除，並醫醫師指示返院追蹤檢查。\n\n5. 若平時有服用抗血栓藥物（心臟開刀或梗塞型中風病患）、阿斯匹靈、或本身為凝血異常病患（血友病、肝硬化或肝癌、造血功能異常者）、或頭頸部癌症病患（如鼻咽癌），如遇無法止血情形，請即速就醫檢查。\n\n義大醫療財團法人關心您'), 0.36574446793301796), (Document(id='fdc29b9a-cfbe-4d70-921b-5fc52920000